# Manditory Requirements

In [1]:
import pandas as pd 
import requests
import re
from pymongo import MongoClient

def send_http_request(url: str, body=None, method="POST"):
    try:
        response = requests.request(method, url, json=body)
        # Check if the request was successful (status code 200)        
        if response.status_code == 200:
            content_type = response.headers.get('Content-Type')
            if content_type and 'application/json' in content_type:
                try:
                    return response.json()
                except requests.exceptions.JSONDecodeError as e1:
                    return response.text
            else:
                return response.text

        else:
            print(f"Request exited with status code {response.status_code}: {response.reason}")
    except requests.RequestException as e:
        return e
    return None

Constant mapping between exercise names and their root derivation trees:

In [2]:
name_to_model_ids = {
    'Courses': ['JDKw8yJZF5fiP3jv3', 'PSqwzYAfW9dFAa9im'],
    'ProductionLine_v2_v3': ['aTwuoJgesSd8hXXEP', 'bNCCf9FMRZoxqobfX'],
    'Train': ['QxGnrFQnXPGh2Lh8C'],
    'SocialNetwork': ['dkZH6HJNQNLLDX6Aj'],
    'TrashFOL': ['sDLK7uBCbgZon3znd'],
    'ClassroomFOL': ['YH3ANm7Y5Qe5dSYem'],
    'TrashRL': ['PQAJE67kz8w5NWJuM'],
    'ClassroomRL': ['zRAn69AocpkmxXZnW'],
    'Graphs': ['gAeD3MTGCCv8YNTaK'],
    'LTS': ['zoEADeCW2b2suJB2k'],
    'ProductionLine_v1': ['jyS8Bmceejj9pLbTW'],
    'CV': ['JC8Tij8o8GZb99gEJ'],
    'TrashLTL': ['9jPK8KBWzjFmBx4Hb']
}

# Setup Databases

Setup databases for SpecAssistant and HiGenA.

## Setup HiGenA

In [ ]:
for (name,ids) in name_to_model_ids.items():
    send_http_request(url="http://localhost:8080/hint/higena-setup",  body=ids)

## Setup SpecAssistant

Optional SpecAssistant Database Drop

In [4]:
send_http_request(url="http://localhost:8080/hint/debug-drop-db", method="GET")

Request exited with status code 204: No Content


In [5]:
def send_specassistant_setup_from_name(name):
    if (name in name_to_model_ids):
        return send_http_request(url="http://localhost:8080/hint/specassistant-setup?prefix="+name, body=name_to_model_ids[name], method="GET")
    else:
        return "Unkown Exercise"

### Setup Single

#### Setup Classroom

In [ ]:
# FOL
send_specassistant_setup_from_name("ClassroomFOL")

In [ ]:
# RL
send_specassistant_setup_from_name("ClassroomRL")

#### Setup Social Network

In [6]:
# Setup SpecAssistant Desired Graphs
send_specassistant_setup_from_name("SocialNetwork")

'Setup completed for SocialNetwork with model_ids [dkZH6HJNQNLLDX6Aj]'

### Setup All

In [53]:
# WARNING: This takes some time and will run in the foreground, setting up every graph, one at a time
for name in name_to_model_ids.keys():
    print(send_specassistant_setup_from_name(name))

Setup completed for Courses with model_ids [JDKw8yJZF5fiP3jv3, PSqwzYAfW9dFAa9im]
Setup completed for ProductionLine_v2_v3 with model_ids [aTwuoJgesSd8hXXEP, bNCCf9FMRZoxqobfX]
Setup completed for Train with model_ids [QxGnrFQnXPGh2Lh8C]
Setup completed for SocialNetwork with model_ids [dkZH6HJNQNLLDX6Aj]
Setup completed for TrashFOL with model_ids [sDLK7uBCbgZon3znd]
Setup completed for ClassroomFOL with model_ids [YH3ANm7Y5Qe5dSYem]
Setup completed for TrashRL with model_ids [PQAJE67kz8w5NWJuM]
Setup completed for ClassroomRL with model_ids [zRAn69AocpkmxXZnW]
Setup completed for Graphs with model_ids [gAeD3MTGCCv8YNTaK]
Setup completed for LTS with model_ids [zoEADeCW2b2suJB2k]
Setup completed for ProductionLine_v1 with model_ids [jyS8Bmceejj9pLbTW]
Setup completed for CV with model_ids [JC8Tij8o8GZb99gEJ]
Setup completed for TrashLTL with model_ids [9jPK8KBWzjFmBx4Hb]


# Database Study

In [78]:
mongo_uri = "mongodb://localhost:27017/"
database_name = "meteor"

## Data Gathering

### Get GraphId Maps

In [95]:
def get_graph_id_dict_pipeline():
    """Targets "Graph" collection"""
    return [
        {'$addFields': {'reg': {'$regexFind': {'input': '$name', 'regex': re.compile(r"([^-]*)-(.*)")}}}}, # Apply a regex find using the provided regex
        {'$addFields': {'super_name': {'$first': '$reg.captures'}}}, # Place the first regex group on field super_name
        {'$group': {'_id': '$super_name', 'graph_ids': {'$push': '$_id'}}} # Group based on super_name, accumulate the graph ids in the arrays graph_ids
    ]

client = MongoClient(mongo_uri)
db = client[database_name]

graph_collection = db["Graph"]

name_to_graph_ids = {} 

for doc in graph_collection.aggregate(get_graph_id_dict_pipeline()):
    name_to_graph_ids[doc["_id"]] = doc["graph_ids"]

client.close()

### Get Graph Stats Data Frames

In [96]:
def get_graph_node_statistics():
    """Targets "Node" collection"""
    return [
        {'$group': { # Group
            '_id': '$graph_id', #By graph_id
            'valid_nodes': {'$sum': {'$cond': ['$valid', 1, 0]}}, # Count valids
            'invalid_nodes': {'$sum': {'$cond': ['$valid', 0, 1]}},  # Count invalids
            'valid_submissions': {'$sum': {'$cond': ['$valid', '$visits', 0]}}, # Sum valid frequencies
            'invalid_submissions': {'$sum': {'$cond': ['$valid', 0, '$visits']}} # Sum invalid frequencies
        }},
        {'$lookup': {'from': 'Graph', 'localField': '_id', 'foreignField': '_id', 'as': 'graph'}}, # Lookup the graph's specification and place it in an array
        {'$unwind': '$graph'}, # Unwind the graph array
        {'$project': {  # Rewrite the object as the defined fields
            '_id': 0, # means remove id field
            'name': '$graph.name', 
            'valid_formulas': '$valid_nodes', 
            'invalid_formulas': '$invalid_nodes', 
            'valid_submissions': '$valid_submissions', 
            'invalid_submissions': '$invalid_submissions'
        }},
        {'$sort': {'name': 1}} # Sort from biggest to smallest based on the name
    ]

client = MongoClient(mongo_uri)
db = client[database_name]

node_collection = db["Node"]

data = list(node_collection.aggregate(get_graph_node_statistics()))

graph_stats_df = pd.DataFrame(data)

client.close()

### Get Popular Node Data Frames

WARNING: Requires GraphId Maps

In [97]:
def get_popular_nodes_pipeline(graph_ids):
    """Targets "Node" collection"""
    return [
        {'$match': {'valid': False, 'graph_id': {"$in":graph_ids}}}, # Match invalid nodes with one of the provided graph ids
        {'$addFields': {'formula': {'$objectToArray': '$formula'}}}, # Covert the formula object into an array of key value objects.
        {'$unwind': '$formula'}, # Unwind object the formula (since every array is a singleton this will just update each document)
        {'$match': {'formula.v': {'$ne': ''}}}, # Match non blank formulas (aka remove initial nodes)
        {'$lookup': {'from': 'Node', 'localField': 'minSolution', 'foreignField': '_id', 'as': 'minSolutionFormula', 
          # A "join" with the document Node, values are placed in the array minSolutionFormula
            'pipeline': [ # Pipeline applied to the "joined" document
                {'$addFields': {'formula': {'$objectToArray': '$formula'}}}, # Covert the formula object into an array of key value objects.
                {'$unwind': '$formula'}, # Unwind object the formula (since every array is a singleton this will just update each document)
                {'$replaceRoot': {'newRoot': '$formula'}} # replace the object with its formula key value object (k,v)
            ]
        }},
        {'$unwind': '$minSolutionFormula'}, # Unwind the minSolutionFormula array (since only one object can be present there wont be any new documents)
        {'$project': { # Rewrite the object as the defined fields
            '_id':0, # means remove id field
            'predicate': '$formula.k', 
            'formula': '$formula.v',
            'frequency': '$visits',
            'closest_solution': '$minSolutionFormula.v', 
            'closest_solution_edit_distance': '$minSolutionTed'
        }},
        {'$sort': {'frequency': -1}} # Sort from biggest to smallest based on the defined frequency
    ]

client = MongoClient(mongo_uri)
db = client[database_name]

node_collection = db["Node"]

name_to_pop_dfs = {}

for (name,graph_ids) in name_to_graph_ids.items():
    data = list(node_collection.aggregate(get_popular_nodes_pipeline(graph_ids)))[0:30] # Limits output to first 30 entries
    df_ = pd.DataFrame(data)
    name_to_pop_dfs[name] = df_

client.close()

### Get Min Solutions Data Frames

WARNING: Requires GraphId Maps

In [98]:
def get_min_solutions_pipeline(graph_ids):
    """Targets "Node" collection"""
    return [
        {'$match': { 'graph_id': {"$in":graph_ids}}}, # Match nodes with one of the provided graph ids
        {'$group': {'_id': '$minSolution','count': {'$sum': 1}}}, # Count the minimum solution frequency
        {'$lookup': {'from': 'Node', 'localField': '_id', 'foreignField': '_id', 'as': 'node', 
            # A "join" with the document Node, values are placed in the array minSolutionFormula
            'pipeline': [ # Pipeline applied to the "joined" document
                {'$addFields': {'formula': {'$objectToArray': '$formula'}}}, # Covert the formula object into an array of key value objects.
                {'$unwind': '$formula'}, # Unwind object the formula (since every array is a singleton this will just update each document)
            ]
        }},
        {'$unwind': '$node'},
        {'$project': { # Rewrite the object as the defined fields
            '_id': 0,  # means remove id field
            'predicate': '$node.formula.k', 
            'formula': '$node.formula.v', 
            'frequency': '$node.visits',
            'frequency_as_the_closest_solution':'$count'
        }},
        {'$sort': {'frequency': -1}} # Sort from biggest to smallest based on the defined frequency
    ]

client = MongoClient(mongo_uri)
db = client[database_name]

node_collection = db["Node"]

name_to_min_sol_dfs = {}

for (name,graph_ids) in name_to_graph_ids.items():
    data = list(node_collection.aggregate(get_min_solutions_pipeline(graph_ids)))
    df_ = pd.DataFrame(data)
    name_to_min_sol_dfs[name] = df_

client.close()

## Data Frame Persistance

### Write As Multiple Csvs

General Statistics

In [ ]:
graph_stats_df.to_csv(path_or_buf="graph_stats.csv",sep=';',float_format='%g',mode='w', index=False)

Popular Formulas

In [14]:
for (name, df_) in name_to_pop_dfs.items():
    df_.to_csv(path_or_buf=name+".popularity.csv",sep=';',float_format='%g',mode='w', index=False)

Solution Formulas

In [ ]:
for (name, df_) in name_to_min_sol_dfs.items():
    df_.to_csv(path_or_buf=name+".solution.csv",sep=';',float_format='%g',mode='w', index=False)

### Write as Sheets of a Single XLSX File

WARNING: Requires Every DataFrame

In [99]:
import xlsxwriter

with pd.ExcelWriter('db_study.xlsx', engine='xlsxwriter') as writer:
    workbook = writer.book
    text_wrap = workbook.add_format({'text_wrap': True, 'valign': 'top'})
    bold = workbook.add_format({'bold': True})
    for name in sorted(list(name_to_model_ids.keys())):
        sheet = workbook.add_worksheet(name=name)

        sheet.set_column(0,0,15)
        sheet.set_column(1,1,100,text_wrap)
        sheet.set_column(2,2,15)
        sheet.set_column(3,3,100,text_wrap)
        sheet.set_column(4,4,27)
        
        row = 0
        sheet.merge_range(row,0,row,len(name_to_pop_dfs[name]),"The 30 most frequent formulas",bold)
        row+=1
        name_to_pop_dfs[name].to_excel(excel_writer=writer,sheet_name=name,startrow=row, index=False)
        row+= name_to_pop_dfs[name].shape[0] + 2
        sheet.merge_range(row,0,row,len(name_to_min_sol_dfs[name]),"The valid formulas ordered by their frequency",bold)
        row+=1
        name_to_min_sol_dfs[name].to_excel(excel_writer=writer,sheet_name=name,startrow=row, index=False)
        row+= name_to_min_sol_dfs[name].shape[0] + 2
    
    graph_stats_df.to_excel(excel_writer=writer,sheet_name="General Statistics", index=False)
    workbook.get_worksheet_by_name('General Statistics').set_column(0,0,30)
    workbook.get_worksheet_by_name('General Statistics').set_column(1,4,20)

# Policy Study

In [48]:
exercise = "dkZH6HJNQNLLDX6Aj"
data = pd.read_csv('formulas.csv', delimiter=';')

input_data = None
input_data = pd.read_csv('formulas.csv', delimiter=';').sort_values(by='Predicate')
input_data = input_data.reset_index()

body = dict()

for index, row in input_data.iterrows():
    try:
        body[row['Predicate']].append(row['Formula'])
    except KeyError:
        body[row['Predicate']] = [row['Formula']]

input_data

,index,Predicate,Formula
0,0,inv1,all p: Photo |some u: User | u -> p in posts
1,1,inv1,all p: Photo | p in User . posts
2,2,inv1,"all p: Photo, u: User | p in u . posts"
3,3,inv5,"all i: Influencer, u: User | i in u . follows"
4,4,inv5,all u: User | Influencer in u . follows
5,5,inv5,all u: User | u . follows in Influencer


In [54]:
output=requests.request("POST", "http://localhost:8080/study/test-all-policies-on-formulas?model_id="+exercise, json=body).json()
print("OK")

OK


In [ ]:
df = pd.DataFrame(output)
df

In [56]:
condensed_output = dict()
for obj in output:
    try:
        condensed_output[(obj['predicate'],obj['formula'],obj.get('nextFormula', None))][obj['policy']] = True
    except KeyError:
        copy = dict(obj)
        policy = obj['policy']
        copy.pop('policy',None)
        copy[policy] = True
        condensed_output[(obj['predicate'],obj['formula'],obj['nextFormula'])] =copy
    
df = pd.DataFrame(condensed_output.values())
df

,formula,predicate,hint,nextFormula,normalizedFormula,MAX-FREQ,COMPxPOPULARITY,TEDxPOPULARITY,BALANCED-TEDCOMP,MINMAX-TED,...,MIN-TED,TEDxArrival,BALANCED-TEDCOMPxPOPULARITY,MIN-ONE,MINMAX-COMP,MAXIMIN-FREQ,Arrival,POPULARITY,BALANCED-TEDCOMPxArrival,COMPxArrival
0,all p: Photo |some u: User | u -> p in posts,inv1,One step away from the solution! Instead of us...,(all ref0:(one Photo)|(one ref1:(one User)|((r...,(all ref0:(one Photo)|(some ref1:(one User)|((...,True,True,True,True,True,...,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN
1,all p: Photo | p in User . posts,inv1,"Keep going! Instead of using var0, try using t...",,(all ref0:(one Photo)|(ref0 in (User . (User <...,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN
2,"all p: Photo, u: User | p in u . posts",inv1,Keep going! It seems like the inclusion operat...,(all ref0:(one Photo)|(ref0 in (User . (User <...,"(all ref0:(one Photo),ref1:(one User)|(ref0 in...",True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"all i: Influencer, u: User | i in u . follows",inv5,Near a solution! Instead of using variable of ...,"(all ref0:(one Influencer),ref1:(one User)|(re...","(all ref0:(one Influencer),ref1:(one User)|(re...",True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,all u: User | Influencer in u . follows,inv5,Near a solution! Consider adding a difference ...,(all ref0:(one User)|((Influencer - ref0) in (...,(all ref0:(one User)|(Influencer in (ref0 . (U...,True,True,True,True,True,...,NaN,True,True,True,True,True,True,True,NaN,NaN
5,all u: User | u . follows in Influencer,inv5,Keep going! You can use variables to help spec...,"(all ref0:(one Influencer),ref1:(one User)|((r...",(all ref0:(one User)|((ref0 . (User <: follows...,True,NaN,NaN,True,NaN,...,NaN,NaN,NaN,True,True,True,NaN,True,True,True
6,all p: Photo | p in User . posts,inv1,Keep going! It seems like the inclusion operat...,(Photo in (User . (User <: posts))),(all ref0:(one Photo)|(ref0 in (User . (User <...,NaN,True,True,NaN,NaN,...,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"all p: Photo, u: User | p in u . posts",inv1,One step away from the solution! Instead of us...,(all ref0:(one Photo)|(one ref1:(one User)|(re...,"(all ref0:(one Photo),ref1:(one User)|(ref0 in...",NaN,True,True,NaN,True,...,True,True,True,NaN,True,NaN,NaN,NaN,NaN,NaN
8,"all i: Influencer, u: User | i in u . follows",inv5,Near a solution! Consider adding a difference ...,"(all ref0:(one Influencer),ref1:(one (User - r...","(all ref0:(one Influencer),ref1:(one User)|(re...",NaN,True,True,NaN,True,...,True,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,all u: User | u . follows in Influencer,inv5,Near a solution! Consider adding a signature o...,(all ref0:(one User)|(Influencer in (ref0 . (U...,(all ref0:(one User)|((ref0 . (User <: follows...,NaN,True,True,NaN,True,...,True,True,True,NaN,NaN,NaN,True,NaN,NaN,NaN


In [57]:
df.sort_values(by='predicate', ascending=True)
df.to_csv("hints.csv", index=False, sep=";")


# Request hints

Import data from file. Checks out the top 10 incorrect most popular submissions  and stores 
the predicates they belong to. Then, it creates a new dataframe with the top 3 most popular submissions for each one of these predicates.

In [ ]:
# setting the display options
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', None)

# Get the predicates with the most popular incorrect answers 
challenge = "dkZH6HJNQNLLDX6Aj"
data = pd.read_csv('popularity.csv', delimiter=';').sort_values(by='Popularity', ascending=False)
predicates = data.head(10)["Predicate"].unique().tolist()

df = pd.DataFrame()
for predicate in predicates:
    top = data[data["Predicate"] == predicate].head(3)
    df = pd.concat([df, top], ignore_index=True)

df.drop(columns=['Popularity'], inplace=True)
df

WARNING: Run only after setup is complete.

In [ ]:
def gen_body_request(model, challenge, predicate, expression, hintGenType):
    obj = {
        "model": model + " pred " + predicate + " { " + expression + " }",
        "challenge": challenge,
        "predicate": predicate,
        "hintGenType": hintGenType
    }

    return obj

def get_hint(model, url, challenge, predicate, expression, hintGenType = "TED"):
    body = gen_body_request(model, challenge, predicate, expression, hintGenType)
    response = send_http_request(url, body)

    if response is not None:
        return pd.Series([response["hint"], response["nextExpr"], response["targetExpr"]])
    else:
        return pd.Series(["", "", ""])

## HiGenA
### TED policy
Generates hints using HiGenA with the path with the lowest TED.

In [ ]:
model = "sig User {follows : set User,sees : set Photo,posts : set Photo,suggested : set User} sig Influencer extends User {} sig Photo {date : one Day} sig Ad extends Photo {} sig Day {}"
url = "http://localhost:8080/hint/higena-hint"

# Hint using Higena with TED
higenaTED = df.copy()
columns = df.apply(lambda row: get_hint(model, url, row["Challenge"], row["Predicate"], row["Expression"]), axis=1)
higenaTED[["hint", "next", "solution"]] = columns

higenaTED

### Node Popularity policy

Generates hints using HiGenA with the path with the most popular submissions.

In [ ]:
# Hint using Higena with Most popular submissions
higenaNode = df.copy()
columns = df.apply(lambda row: get_hint(model, url, row["Challenge"], row["Predicate"], row["Expression"], hintGenType="NODE_POISSON"), axis=1)
higenaNode[["hint", "next", "solution"]] = columns

higenaNode

### Edge Popularity policy
Generates hints using HiGenA with the path with the most popular transitions.

In [ ]:
# Hint using Higena with Most popular submissions
higenaEdge = df.copy()
columns = df.apply(lambda row: get_hint(model, url, row["Challenge"], row["Predicate"], row["Expression"], hintGenType="REL_POISSON"), axis=1)
higenaEdge[["hint", "next", "solution"]] = columns

higenaEdge

## Spec Assistant
### Default policy
Generates hints using SpecAssistant default parameters.

In [ ]:
# Compute policy
url = "http://localhost:8080/hint/compute-all-policies-for-rule?rule=TEDCOMPXxArrival"
send_http_request(url, [challenge], "POST")

In [ ]:
# Hint using Spec Assistant default parameters
url = "http://localhost:8080/hint/spec-hint"
spec = df.copy()
columns = df.apply(lambda row: get_hint(model, url, row["Challenge"], row["Predicate"], row["Expression"]), axis=1)
spec[["hint", "next", "solution"]] = columns
spec

### TED policy

In [ ]:
# Compute policy
url = "http://localhost:8080/hint/compute-all-policies-for-rule?rule=TED"
send_http_request(url)

In [ ]:
# Generate hint
url = "http://localhost:8080/hint/spec-hint"
specTed = df.copy()
columns = df.apply(lambda row: get_hint(model, url, row["Challenge"], row["Predicate"], row["Expression"]), axis=1)
specTed[["hint", "next", "solution"]] = columns
specTed

### Node Popularity policy

In [ ]:
# Compute policy
url = "http://localhost:8080/hint/compute-all-policies-for-rule?rule=MAXFREQ"
send_http_request(url)

In [ ]:
# Generate hint
url = "http://localhost:8080/hint/spec-hint"
specPopularNode = df.copy()
columns = df.apply(lambda row: get_hint(model, url, row["Challenge"], row["Predicate"], row["Expression"]), axis=1)
specPopularNode[["hint", "next", "solution"]] = columns
specPopularNode

### Edge Popularity policy

In [ ]:
# Compute policy
url = "http://localhost:8080/hint/compute-all-policies-for-rule?rule=POPULARITY"
send_http_request(url)

In [ ]:
# Generate hint
url = "http://localhost:8080/hint/spec-hint"
specPopularEdge = df.copy()
columns = df.apply(lambda row: get_hint(model, url, row["Challenge"], row["Predicate"], row["Expression"]), axis=1)
specPopularEdge[["hint", "next", "solution"]] = columns
specPopularEdge

# Export

Export hints to file.

In [ ]:
df["HiGenATED"] = higenaTED["hint"]
df["HiGenAPopularNode"] = higenaNode["hint"]
df["HiGenAPopularEdge"] = higenaEdge["hint"]
df["specTed"] = specTed["hint"]
df["specPopularNode"] = specPopularNode["hint"]
df["specPopularEdge"] = specPopularEdge["hint"]
# Export hints csv
df.to_csv("hints.csv", index=False, sep=";")


In [ ]:
df